In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv("C:\\Code\\Comment Toxicity\\Data\\train.csv\\train.csv")

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
from tensorflow.keras.layers import TextVectorization

In [5]:
X = df['comment_text']
Y = df[df.columns[2:]].values

In [6]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [7]:
df['comment_text']

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [8]:
MAX_FEATURES = 200000

In [9]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [10]:
vectorizer.adapt(X.values)

In [11]:
vectorizer('Hello')[:1]

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([288], dtype=int64)>

In [12]:
vectorized_text = vectorizer(X.values)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [14]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [15]:
train_generator = train.as_numpy_iterator().next()

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [17]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [18]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history = model.fit(train, epochs=10, validation_data=val)

In [21]:
history.history


{'loss': [0.0636649951338768], 'val_loss': [0.04430970549583435]}

In [23]:
input_text = vectorizer('You freaking suck!')

In [25]:
res = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 2s 2s/step


In [26]:
(res > 0.5).astype(int)

array([[1, 0, 1, 0, 1, 0]])

In [27]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [28]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 471ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [29]:
res.shape

(1, 6)

In [30]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [31]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [32]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 150ms/step


In [33]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')


Precision: 0.8117443919181824, Recall:0.6752873659133911, Accuracy:0.48846539855003357


In [34]:
!pip install gradio jinja2

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/57.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/57.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/57.8 MB ? eta -:--:--
    --------------------------------------- 0.8/57.8 MB 1.2 MB/s eta 0:00:46
    --------------------------------------- 1.0/57.8 MB 1.4 MB/s eta 0:00:42
    --------------------------------------- 1.3/57.8 MB 1.5 MB/s eta 0:00:38
   - -------------------------------------- 2.1/57.8 MB 1.7 MB/s eta 0:00:34
   - -------------------------------------- 2.6/57.8 MB 1.9 MB/s eta 0:00:30
   -- ------------------------------------- 3.1/57.8 MB 1.9 MB/s eta 0:00:30
   -- ------------------------------------- 3.7/57.8 MB 2.0 MB/s eta 0:00:27
   -- ------------------------------------- 4.2/57.8 MB 2.1 MB/s eta 0:00:26
   --- ------------------------------------ 5.0/57.8 MB 2.2 MB/s eta 0:00:24
   --- ------------


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------- ----------------------- 24.1/57.8 MB 3.6 MB/s eta 0:00:10
   ----------------- ---------------------- 24.9/57.8 MB 3.7 MB/s eta 0:00:10
   ------------------ --------------------- 26.2/57.8 MB 3.7 MB/s eta 0:00:09
   ------------------ --------------------- 27.0/57.8 MB 3.7 MB/s eta 0:00:09
   ------------------- -------------------- 27.8/57.8 MB 3.7 MB/s eta 0:00:09
   -------------------- ------------------- 29.1/57.8 MB 3.8 MB/s eta 0:00:08
   -------------------- ------------------- 29.6/57.8 MB 3.7 MB/s eta 0:00:08
   --------------------- ------------------ 30.4/57.8 MB 3.7 MB/s eta 0:00:08
   --------------------- ------------------ 31.5/57.8 MB 3.8 MB/s eta 0:00:07
   ---------------------- ----------------- 32.8/57.8 MB 3.8 MB/s eta 0:00:07
   ----------------------- ---------------- 33.6/57.8 MB 3.8 MB/s eta 0:00:07
   ----------------------- ---------------- 34.6/57.8 MB 3.9 MB/s eta 0:00:06
   ------------------------ --------------- 35.7/57.8 MB 3.9 MB/

In [35]:
import tensorflow as tf
import gradio as gr

In [36]:
model.save('toxicity.h5')

C:\Users\vinay\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
model = tf.keras.models.load_model('toxicity.h5')

In [38]:
input_str = vectorizer('hey i freaken hate you!')

In [39]:
res = model.predict(np.expand_dims(input_str,0))


1/1 [==============================] - 2s 2s/step


In [40]:

res

array([[0.7558601 , 0.01089748, 0.31355524, 0.01370266, 0.29245734,
        0.0553227 ]], dtype=float32)

In [41]:

def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [45]:
import gradio as gr

interface = gr.Interface(
    fn=score_comment, 
    inputs=gr.Textbox(lines=2, placeholder='Comment to score'),  # Remove `gr.input`
    outputs="text"
)




In [46]:

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


1/1 [==============================] - 0s 178ms/step
